In [ ]:
import numpy as np
import pandas as pd
from math import log, exp, sqrt 
from sklearn.preprocessing import MinMaxScaler
from pickle import load
from keras.models import load_model
import os
from sklearn.calibration import CalibratedClassifierCV

In [ ]:
def imp_features(data_path):
  folder_name= "/content/drive/My Drive/Self Case Study 1/data/"
  path = os.path.join(folder_name,data_path) 
  index_columns_names =  ["Engine_No","Time_in_cycles"] 
  operational_settings_columns_names = ['Altitude','Mach_number','TRA'] 
  sensor_measure_columns_names = ['T2','T24','T30', 'T50', 'P2','P15','P30', 'Nf','Nc','epr','Ps30','phi','NRf','NRc','BPR','fuel_air_ratio','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']
  input_file_column_names = index_columns_names + operational_settings_columns_names + sensor_measure_columns_names
  machine_df= pd.read_csv(path,delim_whitespace=True,names=input_file_column_names) 
  imp_col = ['Engine_No','T50','Nf','Ps30','phi','NRf','NRc','BPR','W31','W32','T48','T41','T90','Ve','EGT_margin','Nc/Nf','PCNcRdmd ','M_cold','W_f','Thrust','Fan_thrust','core_thrust','TSFC','Thermal_efficiency']
  ###############################################
  cp_gas = 1147 #specific heat of gas
  cp_air = 1005 # #specific heat of air
  transmission_efficiency = 0.985 # of high pressure compressor 
  gamma_gas = 1.33
  gamma_air = 1.4    #  specific heats of air
  nozzle_efficiency =0.95  # efficiency of nozzle
  fan_efficiency =0.99    #  efficiency of turbo fan
  nozzle_efficiency = 0.995 # nozzle fan efficiency 
  R = 287  # gas constant
  machine_df['Altitude']= machine_df['Altitude'].apply(lambda x: x*1000*0.3048) # convert killo feet to meters 
  machine_df['T2'] =  machine_df['T2'].apply(lambda x: x*0.555556)  #convert temperature Rankine to Kelvin 
  machine_df['P2']  = machine_df['P2'].apply(lambda x: x*0.0689476) # convert pressure psia to bar
  machine_df['T24'] = machine_df['T24'].apply(lambda x: x*0.555556) #convert temperature Rankine to Kelvin
  machine_df['T30'] = machine_df['T30'].apply(lambda x: x*0.555556) #convert temperature Rankine to Kelvin
  machine_df['T50'] = machine_df['T50'].apply(lambda x: x*0.555556) #convert temperature Rankine to Kelvin 
  machine_df['Ps30'] = machine_df['Ps30'].apply(lambda x: x*0.0689476) # convert pressure to bar 
  #######################################################
  #######################################################
  # calc High pressure turbine outlet temperature 
  def HPT_outlet(T24,T2,T50,BPR):
    T48= T50+(cp_air*(T24-T2)*(BPR+1))/(cp_gas*transmission_efficiency)
    return T48 
  machine_df['T48'] = machine_df.apply(lambda row :HPT_outlet(row['T24'],row['T2'],row['T50'],row['BPR']),axis= 1) 
  ######################################################
  #Turbine entry temperature 
  def turbin_inlet(T48,T30,T24):
    T41 = T48+(cp_air*(T30-T24))/(cp_gas*transmission_efficiency)
    return T41 
  machine_df['T41']= machine_df.apply(lambda row: turbin_inlet(row['T48'],row['T30'],row['T24']),axis = 1) 
  machine_df['P50'] = machine_df['epr']*machine_df['P2']
  ######################################################
  # calculate pressure,temperature and density of an aircraft at its flying altitudes.
  def isa(altitude):
    # troposphere
    if altitude<=11000:
      if altitude == 0:
        temperature =    228.15
        pressure    =    1.01325
        density    =     1.225 
      else:
        temperature = 288.15 - 0.0065 * altitude
        pressure =   1.01325 * (temperature/288.15)**5.25588
        density  =   (pressure/(287.05287*temperature))*10**5
    # stratosphere
    elif altitude <=20000:
      temperature =        288.15 - 0.0065 *11000
      pressure_11000    =  1.01325 * (temperature/288.15)**5.25588
      density_11000     =  (pressure_11000/(287.05287*temperature))*10**5
      ratio             =  exp((-9.80665*(altitude-11000))/(287.05287*temperature))
      pressure          =  pressure_11000*ratio
      density           =  density_11000*ratio
    else:
      raise ValueError('altitude out of range [0-20000m]')
    return  pressure,temperature,density 

  machine_df[['Atm_pressure','Atm_temp','Atm_density']] = machine_df.apply(lambda row: isa(row['Altitude']),axis=1,result_type="expand")
  ################################################
  def aircraft_velocity(Mach_number,Atm_temp):
    V_a= Mach_number*(sqrt(gamma_air*R*Atm_temp))
    return V_a 
  machine_df['Va'] = machine_df.apply(lambda row:aircraft_velocity(row['Mach_number'],row['Atm_temp']),axis=1)
  ################################################
  def EGT(P50,T50,atm_pressure):
    R = ((gamma_gas-1)*cp_gas)/gamma_gas 
    nozzle_pressure_ratio = P50/atm_pressure  # Total temperature at LPT outlet/atmospheric pressure 
    # check nozzle choked condition or not 
    choked_ration =  (1- (gamma_gas-1)/((gamma_gas+1)*nozzle_efficiency))**(-gamma_gas/(gamma_gas-1)) # choked_ration = P50/at choked_pressure 
    choked_pressure = P50/choked_ration
    # =============================
    # Nozzle is not chocked 
    if atm_pressure >= choked_pressure:
      isp_ratio = (atm_pressure/P50)**((gamma_gas-1)/gamma_gas) # we need to consider atm_pressure
      T90 = (1- (1- isp_ratio)*nozzle_efficiency)*T50 # Exhaust Gas Temperature  # nozzle_efficiency = (turbine exit temperature -Nozzle exit temperature) / (turbine exit temperature-ideal Nozzle exit temperature)
      exit_velocity = sqrt(2*cp_gas*(T50-T90))
    #==============================
    # Nozzle is chocked case
    else:
      isp_ratio = (choked_pressure/P50)**((gamma_gas-1)/gamma_gas) # nozzle_efficiency = (turbine exit temperature -Nozzle exit temperature) / (turbine exit temperature-ideal Nozzle exit temperature)
      T90 = (1- (1- isp_ratio)*nozzle_efficiency)*T50  # nozzle_efficiency = (turbine exit temperature -Nozzle exit temperature) / (turbine exit temperature-ideal Nozzle exit temperature)
      exit_velocity = sqrt(gamma_gas*R*T90) 
    return T90, exit_velocity
  machine_df[['T90','Ve']] =  machine_df.apply(lambda row: EGT(row['P50'],row['T50'],row['Atm_pressure']),axis = 1,result_type="expand") 
  machine_df['EGT_margin'] =  machine_df['T90'].apply(lambda x: 1223.15-x)
  machine_df['Nc/Nf']      =  machine_df['Nc']/machine_df['Nf']
  machine_df['PCNcRdmd '] = (machine_df.Nc-machine_df.NRf)/machine_df.Nc 
  #######################################
  # Mass of cold air fan outlet
  def Mass_flow (phi,Ps30,fuel_air_ratio,BPR):
    Mass_flow_fuel = phi*Ps30*0.0001259978805556
    Mass_flow_hotair = Mass_flow_fuel/fuel_air_ratio 
    Mass_flow_coldair = BPR*Mass_flow_hotair 
    return Mass_flow_hotair,Mass_flow_coldair 
  machine_df[['M_Hot','M_cold']] = machine_df.apply(lambda row: Mass_flow(row['phi'],row['Ps30'],row['fuel_air_ratio'],row['BPR']),axis=1,result_type="expand")
  ############################################
  def fan_exit_temperature(T2,T24,T48,T50,fuel_air_ratio,BPR):
    T21 = T2+ (((1+fuel_air_ratio)*(T48-T50)*(cp_gas/cp_air)*transmission_efficiency - (T24-T2))/BPR)
    return T21
  machine_df['T21'] = machine_df.apply(lambda row: fan_exit_temperature(row['T2'],row['T24'],row['T48'],row['T50'],row['fuel_air_ratio'],row['BPR']),axis=1) 
  def Fan_pressure_ratio(Atm_pressure,P2, T21,T2):
    ratio= (1+fan_efficiency*((T21/T2)-1))**(gamma_air/(gamma_air-1))
    P21 = ratio*P2
    choked_ration =  (1- (gamma_air-1)/((gamma_air+1)*nozzle_efficiency))**(-gamma_air/(gamma_air-1)) # choked_ration = P21/at choked_pressure  
    choked_pressure = P21/choked_ration
    # =============================
    # Nozzle is not chocked 
    if Atm_pressure	 >= choked_pressure:
      isp_ratio = (Atm_pressure	/P21)**((gamma_air-1)/gamma_air) # we need to consider atm_pressure
      exit_temp_nozzle = (1- (1- isp_ratio)*nozzle_efficiency)*T21 # Exhaust Gas Temperature  # nozzle_efficiency = (bypass nozzle exit temperature -Nozzle exit temperature) / (fan exit temperature-ideal Nozzle exit temperature)
      exit_velocity = sqrt(2*cp_air*(T21-exit_temp_nozzle))
    #==============================
    # Nozzle is chocked case 
    else:
      isp_ratio = (choked_pressure/P21)**((gamma_air-1)/gamma_air) # nozzle_efficiency = (bypass nozzle exit temperature -Nozzle exit temperature) / (fan exit temperature-ideal Nozzle exit temperature)
      exit_temp_nozzle = (1- (1- isp_ratio)*nozzle_efficiency)*T21  # nozzle_efficiency = (turbine exit temperature -Nozzle exit temperature) / (turbine exit temperature-ideal Nozzle exit temperature)
      exit_velocity = sqrt(gamma_air*R*exit_temp_nozzle) 
    return P21,exit_velocity 
  machine_df[['P21','V_bypass']] =  machine_df.apply(lambda row:  Fan_pressure_ratio(row['Atm_pressure'],row['P2'],row['T21'],row['T2']),axis=1,result_type="expand") 
  def Corrected_flow(M_cold,M_Hot,T2,P2,T21,P21):
    atm_temperature =    228.15
    atm_pressure    =    1.01325
    w_f = (M_cold*sqrt(T2/atm_temperature))/(P2/atm_pressure) #fan_corrected_mass_flow 
    return w_f
  machine_df['W_f'] = machine_df.apply(lambda row: Corrected_flow(row['M_cold'],row['M_Hot'],row['T2'],row['P2'],row['T21'],row['P21']),axis=1,result_type="expand")
  ###############################################################
  # Thrust for a turbofan engine
  def thrust(fuel_air_ratio,BPR,M_Hot,V_bypass,Ve,Va):
    Fan_thrust = BPR*M_Hot*V_bypass - BPR*M_Hot*Va
    core_thrust = M_Hot*(1+fuel_air_ratio)*Ve - M_Hot*Va 
    Thrust= M_Hot*(1+fuel_air_ratio)*Ve + BPR*M_Hot*V_bypass - (1+BPR)*M_Hot*Va 
    TSFC = (fuel_air_ratio*M_Hot*10**6)/Thrust # TSFC may also be thought of as fuel consumption (grams/second) per unit of thrust (kilonewtons, or kN).
    return Thrust,Fan_thrust,core_thrust,TSFC  
  machine_df[['Thrust','Fan_thrust','core_thrust','TSFC']] = machine_df.apply(lambda row: thrust(row['fuel_air_ratio'],row['BPR'],row['M_Hot'],row['V_bypass'],row['Ve'],row['Va']),axis = 1,result_type="expand")
  ############################################################
  def Thermal_efficiency(fuel_air_ratio,BPR,V_bypass,Ve,Va,Thrust,M_Hot,M_cold):
    Q_R = 42000*1000 # Heat of reaction of the fuel
    Thermal_efficiency = (((1+fuel_air_ratio)*(Ve)**2 + BPR*(V_bypass)**2 - (1+BPR)*(Va)**2)/(2*fuel_air_ratio*Q_R))*100
    return Thermal_efficiency 
  machine_df['Thermal_efficiency']= machine_df.apply(lambda row: Thermal_efficiency(row['fuel_air_ratio'],row['BPR'],row['V_bypass'],row['Ve'],row['Va'],row['Thrust'],row['M_Hot'],row['M_cold']),axis = 1) 
  #############################################################
  machine_df = machine_df.loc[:,imp_col] 
  return machine_df

In [ ]:
def scaling(df,sc_path,sc_model,machine_id):
  #load scaleing file 
  folder_name= "/content/drive/My Drive/Self Case Study 1/data/output/"
  file_path = os.path.join(folder_name,sc_path)
  path_sc = os.path.join(file_path,sc_model) 
  scaler =   load(open(path_sc,'rb')) 
  cols = df.columns.difference(['Engine_No']) 
  norm_train_df = pd.DataFrame(scaler.transform(df[cols]), columns=cols, index=df.index)
  join_df = df[df.columns.difference(cols)].join(norm_train_df) 
  sc_df= join_df.reindex(columns = df.columns) 
  machine_df= sc_df[sc_df['Engine_No'] == machine_id].copy().reset_index(drop=True).drop(['Engine_No'], axis = 1)
  return machine_df,file_path 

In [ ]:
def gen_sequence(df,seq_start,seq_length):
  data_array = df.values  
  num_elements = data_array.shape[0]
  if (seq_start+seq_length)<= num_elements:
    seq = data_array[seq_start:seq_start+seq_length, :]
    return seq.reshape((1, 50,23))
  else:
    print("Only {} Engine life cycle data is available,your are give out of range data {}".format(num_elements,seq_start+seq_length))

# classification model

Probability of Machine fail within 30 days

In [ ]:
def load_ML_model(model_path,model_name):
  path = os.path.join(model_path,model_name)
  # load the model from disk
  loaded_model = load(open(path, 'rb'))
  return loaded_model

In [ ]:
def prob_failure_ml(data_path,sc_path,sc_model,machine_id,time_of_cycle,model_name):
  features= imp_features(data_path) 
  scaling_features,file_path = scaling(features,sc_path,sc_model,machine_id) 
  try:
    #at give some point of time engine fail or not 
    model_test = scaling_features.loc[[time_of_cycle],:] 
  except:
    print("Only {} Engine life cycle data is available,your are give out of range data ".format(scaling_features.shape[0]))
  else:
    model= load_ML_model(file_path,model_name) 
    # make predictions on the test set
    yhat = model.predict_proba(model_test)
    return print('Probability that machine will fail within 30 cycle:',yhat[0][1]*100)

In [ ]:
prob_failure_ml('test_FD001.txt','FD001','scaler.pkl',18,132,'logistic_model_cali.pkl') 

Probability that machine will fail within 30 cycle: 64.32923870204318


# Regression

In [ ]:
def loadmodel(model_path,model_name):
  path = os.path.join(model_path,model_name) 
  def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true))) 
  model = load_model(path,custom_objects={'loss':rmse},compile=False)
  model.compile(loss='mae', optimizer='adam',metrics=[rmse]) 
  return model 

In [ ]:
def RUL(data_path,sc_path,sc_model,machine_id,seq_start,seq_length,model_name):
  features= imp_features(data_path) 
  scaling_features,file_path = scaling(features,sc_path,sc_model,machine_id) 
  try:
    machine_test=gen_sequence(scaling_features,seq_start,seq_length) 
  except NameError:
    raise NameError
  else:
    model = loadmodel(file_path,model_name)
    m_pred= model.predict(machine_test)
    return print('RUL of the machine is {}'.format(m_pred[0][0]))

In [ ]:
#test_FD001.txt: is soures file
# 'FD001': path for scaleing file
# 3: find out RUL for which Engine no
# 76: how many life cycle are completed 
# 50:  number of sequences of data point
# model_cnn_feaegg.h5: model file

RUL('test_FD003.txt','FD003','scaler.pkl',3,76,50,'model_feaegg_LSTM.h5') 

RUL of the machine is 130.02334594726562
